In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir="/home/bharatforge/Desktop/BE aug new data/data-split/train"
val_data_dir="/home/bharatforge/Desktop/BE aug new data/data-split/val"


train_datagen=ImageDataGenerator(#rescale=1./255,
                                 rotation_range=45,
                                 width_shift_range=0.3,
                                 height_shift_range=0.3,
                                 horizontal_flip=True,
                                 fill_mode='nearest')

val_datagen=ImageDataGenerator()#rescale=1./255)

2024-02-29 17:22:40.126795: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 17:22:40.162464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 17:22:40.162491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 17:22:40.163373: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 17:22:40.169124: I tensorflow/core/platform/cpu_feature_guar

In [2]:
img_width,img_height=224,224
batch_size=16
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

val_generator=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=(img_height,img_width),
                                              batch_size=batch_size,
                                              class_mode='categorical')

Found 3498 images belonging to 5 classes.
Found 997 images belonging to 5 classes.


In [3]:
train_class_names = set()
num_train_samples=0
for i in train_generator.filenames:
    train_class_names.add(i.split('/')[0])
    num_train_samples+=1
print(num_train_samples)
train_class_names

3498


{'AMD', 'cataract', 'dia_ret', 'glaucoma', 'normal'}

In [4]:
val_class_names = set()
num_val_samples=0
for i in val_generator.filenames:
    val_class_names.add(i.split('/')[0])
    num_val_samples+=1
print(num_val_samples)
print(val_class_names)

num_classes = len(val_class_names)
num_classes


997
{'AMD', 'normal', 'dia_ret', 'cataract', 'glaucoma'}


5

In [6]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers.experimental import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import numpy as np

checkpoint = ModelCheckpoint("AUG_ensemble.tf",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True, verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,
                          restore_best_weights=True)

epochs = 50
callbacks=[checkpoint, earlystop]

# Load the base models
model1 = load_model('aug_EfficientNetB2.h5')
model2 = load_model('aug_ResNet50.h5')
model3 = load_model('aug_DenseNet201.h5')
model4 = load_model('augnewInceptionV3.h5')

# Remove the output layers from the base models
model1 = Model(inputs=model1.inputs,
               outputs=model1.layers[-2].output,
               name='name_of_model_1')
model2 = Model(inputs=model2.inputs,
               outputs=model2.layers[-2].output,
               name='name_of_model_2')
model3 = Model(inputs=model3.inputs,
               outputs=model3.layers[-2].output,
               name='name_of_model_3')
model4 = Model(inputs=model4.inputs,
               outputs=model4.layers[-2].output,
               name='name_of_model_4')

# Define input layer for original features
input_features = Input(shape=(224, 224, 3), name='input_features')

# Generate predictions from each base model
pred1 = model1(input_features)
pred2 = model2(input_features)
pred3 = model3(input_features)
pred4 = model4(input_features)

# Concatenate the predictions along with the original input features
stacked_outputs = Concatenate()([pred1, pred2, pred3, pred4])

# Add regularization (L2 regularization) to the output layer
stacked_outputs = Dropout(0.5)(stacked_outputs)  # Dropout regularization
stacking_output = Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))(stacked_outputs)

# Define the stacking model architecture
stacking_model = Model(inputs=input_features, outputs=stacking_output, name='stacking_model')

# Compile and train the stacking model
stacking_model.compile(loss='categorical_crossentropy',
                        optimizer=RMSprop(learning_rate=0.001),
                        metrics=['accuracy', tf.keras.metrics.AUC()])

# Train the stacking model with your training data
history = stacking_model.fit(train_generator,
                   steps_per_epoch=num_train_samples//batch_size,
                   epochs=epochs,
                   callbacks=callbacks,
                   validation_data=val_generator,
                   validation_steps=num_val_samples//batch_size)


2024-02-29 17:22:42.083130: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 17:22:42.110092: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 17:22:42.110262: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/50


2024-02-29 17:24:09.824629: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape instacking_model/name_of_model_1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-02-29 17:24:18.207149: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 17:24:18.677117: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-29 17:24:18.743766: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 17:24:23.274263: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f150d1bc140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-29 17:24:23.274289: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (

218/218 [==============================] - ETA: 0s - loss: 1.3423 - accuracy: 0.6554 - auc: 0.8813
Epoch 1: val_loss improved from inf to 7.64654, saving model to AUG_ensemble.tf
INFO:tensorflow:Assets written to: AUG_ensemble.tf/assets


INFO:tensorflow:Assets written to: AUG_ensemble.tf/assets


...

218/218 [==============================] - 416s 2s/step - loss: 0.3812 - accuracy: 0.9078 - auc: 0.9841 - val_loss: 0.2669 - val_accuracy: 0.9304 - val_auc: 0.9925
Epoch 13/50
218/218 [==============================] - ETA: 0s - loss: 0.3695 - accuracy: 0.9072 - auc: 0.9856
Epoch 13: val_loss did not improve from 0.26687
218/218 [==============================] - 272s 1s/step - loss: 0.3695 - accuracy: 0.9072 - auc: 0.9856 - val_loss: 0.3214 - val_accuracy: 0.9073 - val_auc: 0.9906
Epoch 14/50
218/218 [==============================] - ETA: 0s - loss: 0.3278 - accuracy: 0.9187 - auc: 0.9884
Epoch 14: val_loss did not improve from 0.26687
218/218 [==============================] - 271s 1s/step - loss: 0.3278 - accuracy: 0.9187 - auc: 0.9884 - val_loss: 0.6763 - val_accuracy: 0.8740 - val_auc: 0.9640
Epoch 15/50
218/218 [==============================] - ETA: 0s - loss: 0.3802 - accuracy: 0.9087 - auc: 0.9846
Epoch 15: val_loss did not improve from 0.26687
218/218 [======================

INFO:tensorflow:Assets written to: AUG_ensemble.tf/assets


218/218 [==============================] - 419s 2s/step - loss: 0.3426 - accuracy: 0.9161 - auc: 0.9874 - val_loss: 0.2651 - val_accuracy: 0.9284 - val_auc: 0.9939
Epoch 18/50
218/218 [==============================] - ETA: 0s - loss: 0.3024 - accuracy: 0.9248 - auc: 0.9900
Epoch 18: val_loss did not improve from 0.26507
218/218 [==============================] - 276s 1s/step - loss: 0.3024 - accuracy: 0.9248 - auc: 0.9900 - val_loss: 0.3782 - val_accuracy: 0.9315 - val_auc: 0.9907
Epoch 19/50
218/218 [==============================] - ETA: 0s - loss: 0.3330 - accuracy: 0.9268 - auc: 0.9882
Epoch 19: val_loss did not improve from 0.26507
218/218 [==============================] - 272s 1s/step - loss: 0.3330 - accuracy: 0.9268 - auc: 0.9882 - val_loss: 0.5969 - val_accuracy: 0.9153 - val_auc: 0.9804
Epoch 20/50
218/218 [==============================] - ETA: 0s - loss: 0.3326 - accuracy: 0.9222 - auc: 0.9873
Epoch 20: val_loss improved from 0.26507 to 0.20932, saving model to AUG_ensemb

INFO:tensorflow:Assets written to: AUG_ensemble.tf/assets


218/218 [==============================] - 413s 2s/step - loss: 0.3326 - accuracy: 0.9222 - auc: 0.9873 - val_loss: 0.2093 - val_accuracy: 0.9496 - val_auc: 0.9959
Epoch 21/50
218/218 [==============================] - ETA: 0s - loss: 0.2880 - accuracy: 0.9339 - auc: 0.9898
Epoch 21: val_loss did not improve from 0.20932
218/218 [==============================] - 271s 1s/step - loss: 0.2880 - accuracy: 0.9339 - auc: 0.9898 - val_loss: 0.3335 - val_accuracy: 0.9264 - val_auc: 0.9867
Epoch 22/50
218/218 [==============================] - ETA: 0s - loss: 0.3073 - accuracy: 0.9337 - auc: 0.9891
Epoch 22: val_loss did not improve from 0.20932
218/218 [==============================] - 276s 1s/step - loss: 0.3073 - accuracy: 0.9337 - auc: 0.9891 - val_loss: 0.5164 - val_accuracy: 0.8992 - val_auc: 0.9728
Epoch 23/50
218/218 [==============================] - ETA: 0s - loss: 0.2891 - accuracy: 0.9319 - auc: 0.9896
Epoch 23: val_loss did not improve from 0.20932
218/218 [======================

In [19]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelBinarizer

def predict_class(image_path, model, train_generator):
    # Load the image
    img = load_img(image_path, target_size=(224, 224))

    # Convert the image to a numpy array
    img_array = img_to_array(img)

    # Expand the dimensions of the numpy array
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the class of the image
    class_prediction = model.predict(img_array)

    # Get the predicted class index
    class_index = np.argmax(class_prediction)

    # Get the class indices and names
    class_indices = train_generator.class_indices
    class_names = dict((v, k) for k, v in class_indices.items())

    # Get the predicted class name
    class_name = class_names[class_index]

    # Return the predicted class name
    return class_name


In [27]:
user_input = "/home/bharatforge/Desktop/new_data_eye/dataset/dia_ret/dia_ret_126.png"
path = user_input
class_index = predict_class(path, stacking_model, train_generator)
print(class_index)

1/1 [==============================] - 0s 53ms/step
dia_ret


In [22]:
user_input = "/home/bharatforge/Desktop/new_data_eye/dataset/cataract/cataract_265.jpg"
path = user_input
class_index = predict_class(path, stacking_model, train_generator)
print(class_index)

1/1 [==============================] - 0s 56ms/step
cataract


In [23]:
user_input = "/home/bharatforge/Desktop/new_data_eye/dataset/glaucoma/glaucoma_02.jpg"
path = user_input
class_index = predict_class(path, stacking_model, train_generator)
print(class_index)

1/1 [==============================] - 0s 49ms/step
glaucoma


In [26]:
user_input = "/home/bharatforge/Desktop/new_data_eye/dataset/AMD/AMD_232.jpeg"
path = user_input
class_index = predict_class(path, stacking_model, train_generator)
print(class_index)

1/1 [==============================] - 0s 47ms/step
AMD


In [28]:
user_input = "/home/bharatforge/Desktop/new_data_eye/dataset/normal/normal_78.png"
path = user_input
class_index = predict_class(path, stacking_model, train_generator)
print(class_index)

1/1 [==============================] - 0s 49ms/step
normal


In [25]:
loaded_model = stacking_model

from sklearn.metrics import classification_report
import numpy as np
# Assuming you have test data in a separate directory
test_data_dir = "/home/bharatforge/Desktop/BE aug new data/data-split/test"
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical')

# Predict on the test set
y_pred = loaded_model.predict(test_generator, steps=len(test_generator), verbose=1)

# Convert predictions to class labels
y_pred_classes = [np.argmax(pred) for pred in y_pred]
true_classes = test_generator.classes

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(true_classes, y_pred_classes, target_names=class_labels))

Found 505 images belonging to 5 classes.
32/32 [==============================] - 39s 1s/step
Classification Report:
              precision    recall  f1-score   support

         AMD       0.99      0.95      0.97       102
    cataract       1.00      0.98      0.99       102
     dia_ret       0.98      0.97      0.98       110
    glaucoma       0.93      0.96      0.95       102
      normal       0.92      0.97      0.95        89

    accuracy                           0.97       505
   macro avg       0.97      0.97      0.97       505
weighted avg       0.97      0.97      0.97       505

